In [18]:
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import matplotlib.pyplot as plt     # 画图
from gensim import corpora, models
import pyLDAvis     # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis

In [2]:
df = pd.read_excel('text_analysis_weibo_sample.xlsx', index_col = 0)

In [3]:
df.head()

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东


### 语料预处理 

#### 剔除符号与数字

In [4]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['标题/微博内容'][0]
remove_nums(test)[:100]

'国债地产行业重磅利好提振风险偏好期债低开低走国债期货全线收跌年期主力合约跌年期主力合约跌年期主力合约跌三大主力合约均创逾一年收盘新低行情解读公开市场方面央行开展了亿元天期逆回购操作中标利率亿元逆回购到'

#### 分词

In [19]:
# 加载中文停用词词典，可个性化设置
# stopwords = open('stopwords.txt', encoding = 'utf-8').read()

def clean_text(text):
    words = jieba.lcut(text)
    # words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)

test = df['标题/微博内容'][0]
clean_text(test)[:20]

'    国债 ： 地产 行业 重磅 利好'

In [6]:
df['标题/微博内容'] = df['标题/微博内容'].astype(str)
df['微博内容分词'] = df['标题/微博内容'].apply(remove_nums)
df['微博内容分词'] = df['微博内容分词'].apply(clean_text)
df['微博内容分词'] = df['微博内容分词'].apply(lambda x: x.split())
df[:3]

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域,微博内容分词
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他,"[国债, 地产, 行业, 重磅, 利好, 提振, 风险, 偏好, 期债, 低开, 低, 走,..."
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西,"[喜迎, 二十大, 忠诚, 保平安]"
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建,"[注意, 事关, 明日, 教资, 考试, 福建省, 教育, 考试院, 发布, 补充, 公告,..."


### LDA

In [7]:
dictionary = corpora.Dictionary(df['微博内容分词'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['微博内容分词']]     # 根据分词结果创建语料库

In [8]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [9]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.017*"的" + 0.011*"重庆" + 0.008*"了" + 0.008*"年" + 0.007*"亿元"')
(1, '0.019*"的" + 0.016*"月" + 0.012*"万吨" + 0.009*"市场" + 0.008*"日"')
(2, '0.024*"的" + 0.010*"考生" + 0.009*"了" + 0.007*"考场" + 0.006*"在"')
(3, '0.019*"的" + 0.013*"中国" + 0.008*"经济" + 0.007*"刘雨昕" + 0.005*"是"')
(4, '0.045*"的" + 0.011*"在" + 0.010*"电子" + 0.010*"烟" + 0.007*"年"')


In [11]:
df['微博内容分词'].iloc[0][:10]

['国债', '地产', '行业', '重磅', '利好', '提振', '风险', '偏好', '期债', '低开']

In [12]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8834588527679443	 
Topic: 0.019*"的" + 0.016*"月" + 0.012*"万吨" + 0.009*"市场" + 0.008*"日" + 0.007*"库存" + 0.007*"较" + 0.006*"预期" + 0.006*"在" + 0.006*"为"

Score: 0.1161065623164177	 
Topic: 0.045*"的" + 0.011*"在" + 0.010*"电子" + 0.010*"烟" + 0.007*"年" + 0.006*"了" + 0.006*"郑州" + 0.006*"是" + 0.005*"从" + 0.005*"公司"


In [13]:
documents = df['微博内容分词'].values

In [14]:
# Function to infer topics for a document
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

# Print topics for each document
for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(1, 0.8834588), (4, 0.11610666)]

Document 2:
[(0, 0.040123254), (1, 0.8395172), (2, 0.040123194), (3, 0.040132605), (4, 0.04010381)]

Document 3:
[(3, 0.9973755)]

Document 4:
[(4, 0.99933463)]

Document 5:
[(0, 0.050020166), (1, 0.05001583), (2, 0.05002016), (3, 0.7999269), (4, 0.050016947)]

Document 6:
[(0, 0.022296349), (1, 0.0229314), (2, 0.022266826), (3, 0.02234113), (4, 0.91016436)]

Document 7:
[(0, 0.01255861), (1, 0.012572705), (2, 0.9496311), (3, 0.012629368), (4, 0.012608209)]

Document 8:
[(0, 0.100006394), (1, 0.5975695), (2, 0.10096104), (3, 0.10127337), (4, 0.10018966)]

Document 9:
[(3, 0.972109)]

Document 10:
[(4, 0.9993379)]



### 可视化

In [15]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) # 备注：上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
# 显示可视化界面
pyLDAvis.display(lda_vis)

D:\Software\Anaconda\Lib\site-packages\pyLDAvis\_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [42]:
# 导出可视化结果到html
# pyLDAvis.save_html(lda_vis, 'lda_visualization.html')

# 直播数据分析

## 导入数据

In [19]:
import os
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import matplotlib.pyplot as plt     # 画图
from gensim import corpora, models
import pyLDAvis     # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis

In [15]:
# 指定文件夹路径
folder_path = '话术文本'

# 获取指定文件夹下所有的txt文件名
file_names = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

# 初始化一个空的DataFrame来存储文件名和内容
df = pd.DataFrame(columns=['文件名', '文件内容'])

# 循环读取每个txt文件，并将其添加到DataFrame中
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        # 创建一个临时的DataFrame来存储当前文件的文件名和内容
        temp_df = pd.DataFrame({'文件名': [file_name], '文件内容': [content]})
        # 将临时DataFrame添加到主DataFrame中
        df = pd.concat([df, temp_df], ignore_index=True)

# 打印生成的DataFrame
print(df)

           文件名                                               文件内容
0  九阳豆浆纯文本.txt  原本的原汁原味，就它就是就跟那种鲜榨豆浆喝起来口感是一样的，它没有科技，没有狠活的，就喝起来...
1   王饱饱纯文本.txt  这几位宝贝们，关于到身材上的是一定要买的是 3 + 6 号链接可以更快。可以的。为什么让你们...


## 预料预处理

### 剔除数字与符号

In [16]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['文件内容'][0]
remove_nums(test)[:100]

'原本的原汁原味就它就是就跟那种鲜榨豆浆喝起来口感是一样的它没有科技没有狠活的就喝起来是浓浓的豆腐那种香味在里面的号链接我们给大家今天花一份钱能够补充三份营养价值黄豆黑豆有机的像黄豆一些孕妈妈你需要补充'

### 分词

In [38]:
# 加载中文停用词词典，可个性化设置
stopwords = open('stopwords.txt', encoding = 'utf-8').read()
def clean_text(text):
    words = jieba.lcut(text)
    words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)

test = df['文件内容'][0]
clean_text(test)[:20]

'原本 原汁原味 那种 鲜榨 豆浆 喝 口'

In [39]:
df['文件内容'] = df['文件内容'].astype(str)
df['内容分词'] = df['文件内容'].apply(remove_nums)
df['内容分词'] = df['内容分词'].apply(clean_text)
df['内容分词'] = df['内容分词'].apply(lambda x: x.split())
df[:3]

,文件名,文件内容,内容分词
0,九阳豆浆纯文本.txt,原本的原汁原味，就它就是就跟那种鲜榨豆浆喝起来口感是一样的，它没有科技，没有狠活的，就喝起来...,"[原本, 原汁原味, 那种, 鲜榨, 豆浆, 喝, 口感, 科技, 狠活, 喝, 浓浓的, ..."
1,王饱饱纯文本.txt,这几位宝贝们，关于到身材上的是一定要买的是 3 + 6 号链接可以更快。可以的。为什么让你们...,"[几位, 宝贝, 身材, 买, 号, 链接, 快, 喝, 预算, 有限, 买, 一个, 买,..."


## LDA主题模型

In [40]:
dictionary = corpora.Dictionary(df['内容分词'])  # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['内容分词']]     # 根据分词结果创建语料库

In [41]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [42]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.001*"链接" + 0.001*"号" + 0.001*"喝" + 0.001*"杯子" + 0.001*"搅拌"')
(1, '0.022*"链接" + 0.019*"号" + 0.018*"喝" + 0.018*"宝贝" + 0.016*"巴西"')
(2, '0.001*"链接" + 0.001*"号" + 0.001*"喝" + 0.001*"巴西" + 0.001*"宝贝"')
(3, '0.040*"链接" + 0.032*"号" + 0.018*"搅拌" + 0.015*"豆浆" + 0.015*"杯子"')
(4, '0.001*"链接" + 0.001*"号" + 0.001*"喝" + 0.001*"拍" + 0.001*"杯子"')


In [43]:
df['内容分词'].iloc[0][:10]

['原本', '原汁原味', '那种', '鲜榨', '豆浆', '喝', '口感', '科技', '狠活', '喝']

In [44]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.9997575283050537	 
Topic: 0.040*"链接" + 0.032*"号" + 0.018*"搅拌" + 0.015*"豆浆" + 0.015*"杯子" + 0.014*"喝" + 0.014*"送" + 0.013*"拍" + 0.013*"版本" + 0.012*"宝贝"


In [45]:
documents = df['内容分词'].values

In [46]:
# Function to infer topics for a document
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

# Print topics for each document
for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(3, 0.9997575)]

Document 2:
[(1, 0.99973)]



### 可视化

In [47]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) # 备注：上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
# 显示可视化界面
pyLDAvis.display(lda_vis)

D:\Software\Anaconda\Lib\site-packages\pyLDAvis\_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [48]:
# 导出可视化结果到html
# pyLDAvis.save_html(lda_vis, 'lda_visualization.html')